In [1]:
# Import Python Tools:
from IPython.display import display, Latex
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
# import sectionproperties as shape

from ASCE import load_combos

from steel_design.AISC import aisc
steel_density = 0.2833 #pci

from concrete_design import material_properties as conc
from concrete_design.steel_reinforcement import rebar_df as rebar
conc_density = 150.0 #pcf

from masonry_design import material_properties as cmu
from masonry_design.material_properties import NW_CMU #psf

from wood_design.ref_design_values.table_4a.table_4a import table_4a_load
table_4a = table_4a_load(species="DF")

<div style="page-break-after: always;"></div>

#### Case 1

In [1]:
Tu = 3820 #lb-in
Vu = 2281 #lbs
Mu = 90200 #lb-in

In [2]:
T1 = 1/4 #in
S = T1 - 1/32 #in
S

0.21875

In [4]:
t = 5/8 * S
t

0.13671875

In [6]:
φ = 0.75
F_EXX = 70 #ksi
Fnw = 0.60 * F_EXX #ksi

Awe = t * (2*6+2*4) #in^2
Rn = Fnw * Awe #kips

φRn = φ * Rn #kips
φRn

86.1328125